In [11]:
import folium
import subprocess
import webbrowser
import sys
from folium import plugins


json = "/Users/skoebric/Dropbox/GitHub/gtgfoliummap/geometry/wherewework.geojson"

m = folium.Map(width = '100%', height = 800, location = (20,5),zoom_start = 2,
               no_wrap=True,max_bounds=True, min_zoom=2, tiles="MapBox Bright")
plugins.ScrollZoomToggler().add_to(m)
tt = folium.GeoJsonTooltip(['Name','Toolkit'], aliases = ['Country','Toolkits'])
style_function = lambda x: {'fillColor': '#73ad02',
                            'color': '#73ad02',
                            'weight':1,
                            }

folium.GeoJson(json, tooltip = tt, name = 'json', style_function = style_function, highlight_function = style_function).add_to(m)
m.save("/Users/skoebric/Dropbox/GitHub/gtgfoliummap/index.html")
m